# Part 1: Setup - Install Required Libraries


In [1]:
!pip3 install cryptography

In [2]:
try:
    import os
    from datetime import datetime, timedelta
    from cryptography import x509
    from cryptography.x509.oid import NameOID, ExtensionOID
    from cryptography.hazmat.primitives import hashes, serialization
    from cryptography.hazmat.primitives.asymmetric import rsa, padding
    from cryptography.hazmat.backends import default_backend
    import hashlib
    import base64

    print("✅ Libraries imported successfully!")

except ImportError as e:
    print(f"Required module missing: {e.name}. Please install it before running the script.")
    exit(1)

✅ Libraries imported successfully!


# Part 2: QL-Pipeline Generates Key Pair

This simulates QL-Pipeline creating their private/public key pair.

The private key will NEVER be shared with providers.

In [3]:
# Generate private key (4096-bit RSA)

print("\n🔑 Generating QL-Pipeline's private key (4096-bit RSA)...")
ql_private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=default_backend()
)

print(f"✅ Private key generated successfully: {ql_private_key}")


🔑 Generating QL-Pipeline's private key (4096-bit RSA)...
✅ Private key generated successfully: <cryptography.hazmat.bindings._rust.openssl.rsa.RSAPrivateKey object at 0x72846c1ca830>


In [4]:
# Extract public key from private key

ql_public_key = ql_private_key.public_key()

In [5]:
print("✅ Private key generated!")
print("✅ Public key extracted from private key!")
print("\n🔒 CRITICAL: Private key stays with QL-Pipeline FOREVER")
print("📄 Public key will be embedded in certificate and shared")

✅ Private key generated!
✅ Public key extracted from private key!

🔒 CRITICAL: Private key stays with QL-Pipeline FOREVER
📄 Public key will be embedded in certificate and shared


In [6]:
# Show key sizes

private_pem = ql_private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)
public_pem = ql_public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

In [7]:
print(f"\n📊 Private key size: {len(private_pem)} bytes")
print(f"📊 Public key size: {len(public_pem)} bytes")


📊 Private key size: 3272 bytes
📊 Public key size: 800 bytes


# Part 3: QL-Pipeline Creates Self-Signed Certificate

The certificate contains the public key plus identity information.

This is what will be distributed to all providers.

In [8]:
# Create certificate subject (QL-Pipeline's identity)

subject = issuer = x509.Name([
    x509.NameAttribute(NameOID.COUNTRY_NAME, "EU"),
    x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Quality Link"),
    x509.NameAttribute(NameOID.COMMON_NAME, "ql-pipeline.eu"),
])

In [ ]:
# Build certificate

print("\n📄 Creating self-signed certificate...")
ql_certificate = x509.CertificateBuilder().subject_name(
    subject
).issuer_name(
    issuer
).public_key(
    ql_public_key  # PUBLIC KEY
).serial_number(
    x509.random_serial_number()
).not_valid_before(
    datetime.utcnow()
).not_valid_after(
    datetime.utcnow() + timedelta(days=3650)  # Valid 10 years
).add_extension(
    x509.SubjectAlternativeName([
        x509.DNSName("ql-pipeline.eu"),
        x509.DNSName("*.ql-pipeline.eu"),
    ]),
    critical=False,
).add_extension(
    x509.KeyUsage(
        digital_signature=True,
        key_encipherment=True,
        content_commitment=False,
        data_encipherment=False,
        key_agreement=False,
        key_cert_sign=False,
        crl_sign=False,
        encipher_only=False,
        decipher_only=False,
    ),
    critical=True,
).sign(ql_private_key, hashes.SHA256(), default_backend())

print("✅ Certificate created and signed!")


📄 Creating self-signed certificate...
✅ Certificate created and signed!


/tmp/ipykernel_10815/331872790.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow()
/tmp/ipykernel_10815/331872790.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime.utcnow() + timedelta(days=3650)  # Valid 10 years


In [10]:
# Display certificate details

print(f"\n📋 Certificate Details:")
print(f"   Subject: {ql_certificate.subject.rfc4514_string()}")
print(f"   Issuer: {ql_certificate.issuer.rfc4514_string()}")
print(f"   Serial: {ql_certificate.serial_number}")
print(f"   Valid From: {ql_certificate.not_valid_before}")
print(f"   Valid Until: {ql_certificate.not_valid_after}")


📋 Certificate Details:
   Subject: CN=ql-pipeline.eu,O=Quality Link,C=EU
   Issuer: CN=ql-pipeline.eu,O=Quality Link,C=EU
   Serial: 458119471077696106444897947151955470066960215820
   Valid From: 2025-12-23 17:14:25
   Valid Until: 2035-12-21 17:14:25


/tmp/ipykernel_10815/1202777775.py:7: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  print(f"   Valid From: {ql_certificate.not_valid_before}")
/tmp/ipykernel_10815/1202777775.py:8: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  print(f"   Valid Until: {ql_certificate.not_valid_after}")


In [11]:
# Calculate fingerprint

cert_bytes = ql_certificate.public_bytes(serialization.Encoding.DER)
fingerprint = hashlib.sha256(cert_bytes).hexdigest()
print(f"   Fingerprint: SHA256:{fingerprint[:32]}...")

   Fingerprint: SHA256:b873455a035f3c47498ba1a642c5c3f6...


In [12]:
# Save certificate (what gets distributed)

cert_pem = ql_certificate.public_bytes(serialization.Encoding.PEM)
print(f"\n📦 Certificate size: {len(cert_pem)} bytes")
print("\n📤 This certificate will be sent to ALL providers!")


📦 Certificate size: 1891 bytes

📤 This certificate will be sent to ALL providers!


# Part 4: Provider A Receives and Stores Certificate

Provider A receives the certificate via email/portal.

They store it in their trusted client certificates.

In [13]:
# Simulate Provider A receiving the certificate

provider_a_name = "Provider A (University)"
print(f"\n📧 {provider_a_name} receives ql-pipeline-client.crt")



📧 Provider A (University) receives ql-pipeline-client.crt


In [14]:
# Provider extracts public key from certificate

provider_stored_certificate = ql_certificate
provider_stored_public_key = provider_stored_certificate.public_key()

In [15]:
print("✅ Certificate stored in: /etc/ssl/client-certs/ql-pipeline.crt")
print("✅ Public key extracted from certificate")
print(f"✅ Configured: ssl_verify_client on")

✅ Certificate stored in: /etc/ssl/client-certs/ql-pipeline.crt
✅ Public key extracted from certificate
✅ Configured: ssl_verify_client on


In [16]:
# Verify what Provider A has

print(f"\n🔍 What Provider A has stored:")
print(f"   - QL-Pipeline's PUBLIC KEY (from certificate)")
print(f"   - Certificate metadata (CN=ql-pipeline.eu)")
print(f"   - Validity dates")
print(f"\n🔍 What Provider A does NOT have:")
print(f"   - QL-Pipeline's PRIVATE KEY (never transmitted!)")


🔍 What Provider A has stored:
   - QL-Pipeline's PUBLIC KEY (from certificate)
   - Certificate metadata (CN=ql-pipeline.eu)
   - Validity dates

🔍 What Provider A does NOT have:
   - QL-Pipeline's PRIVATE KEY (never transmitted!)


# Part 5: QL-Pipeline Initiates Connection

QL-Pipeline wants to fetch courses from Provider A.

The mTLS handshake begins.

In [17]:
print("\n" + "=" * 60)
print("PHASE 4: MTLS HANDSHAKE - INITIAL CONNECTION")
print("=" * 60)

print("\n🔄 QL-Pipeline initiates connection to Provider A")
print("   Request: GET https://provider-a.edu/api/courses")

# Step 1: TLS Handshake
print("\n1️⃣ ClientHello")
print("   QL-Pipeline → Provider A: 'Let's establish secure connection'")
print("   (Supported TLS versions, cipher suites)")

print("\n2️⃣ ServerHello")
print("   Provider A → QL-Pipeline: 'Agreed, let's use TLS 1.3, AES-256'")

print("\n3️⃣ Server Certificate")
print("   Provider A → QL-Pipeline: [Provider A's certificate]")
print("   QL-Pipeline verifies: ✅ Server is legitimate")


PHASE 4: MTLS HANDSHAKE - INITIAL CONNECTION

🔄 QL-Pipeline initiates connection to Provider A
   Request: GET https://provider-a.edu/api/courses

1️⃣ ClientHello
   QL-Pipeline → Provider A: 'Let's establish secure connection'
   (Supported TLS versions, cipher suites)

2️⃣ ServerHello
   Provider A → QL-Pipeline: 'Agreed, let's use TLS 1.3, AES-256'

3️⃣ Server Certificate
   Provider A → QL-Pipeline: [Provider A's certificate]
   QL-Pipeline verifies: ✅ Server is legitimate


# Part 6: Client Authentication 

This is where mTLS differs from regular TLS.

Provider A asks QL-Pipeline to prove its identity.

In [18]:
print("\n" + "=" * 60)
print("PHASE 5: CLIENT AUTHENTICATION (mTLS)")
print("=" * 60)

print("\n4️⃣ CertificateRequest")
print("   Provider A → QL-Pipeline: 'Show me YOUR certificate!'")

print("\n5️⃣ Client Certificate")
print("   QL-Pipeline → Provider A: [Sends ql-pipeline-client.crt]")


PHASE 5: CLIENT AUTHENTICATION (mTLS)

4️⃣ CertificateRequest
   Provider A → QL-Pipeline: 'Show me YOUR certificate!'

5️⃣ Client Certificate
   QL-Pipeline → Provider A: [Sends ql-pipeline-client.crt]


In [19]:
# Provider A verifies certificate

print("\n🔍 Provider A verifies certificate:")
print("   ✅ Certificate subject: ql-pipeline.eu")
print("   ✅ Certificate not expired")
print("   ✅ Certificate in trusted list")
print("   ✅ Signature valid")


🔍 Provider A verifies certificate:
   ✅ Certificate subject: ql-pipeline.eu
   ✅ Certificate not expired
   ✅ Certificate in trusted list
   ✅ Signature valid


In [20]:
# Now comes the PROOF OF OWNERSHIP challenge

print("\n6️⃣ Challenge-Response (Proof of Private Key Ownership)")
print("   This is the CRITICAL SECURITY STEP!")
print("\n" + "-" * 60)


6️⃣ Challenge-Response (Proof of Private Key Ownership)
   This is the CRITICAL SECURITY STEP!

------------------------------------------------------------


In [21]:
# Provider A generates random challenge

print("\n🎲 Provider A generates random challenge:")

challenge_data = os.urandom(32)  # 32 random bytes
challenge_b64 = base64.b64encode(challenge_data).decode()

print(f"   Challenge (base64): {challenge_b64[:40]}...")
print(f"   Challenge (hex): {challenge_data.hex()[:40]}...")


🎲 Provider A generates random challenge:
   Challenge (base64): aiGYThslC95ErQKFqgcUn4Ys5cIf36RlVpD1pVJy...
   Challenge (hex): 6a21984e1b250bde44ad0285aa07149f862ce5c2...


In [22]:
print("\n📤 Provider A → QL-Pipeline:")
print("   'Sign this random data with your private key to prove you own it!'")


📤 Provider A → QL-Pipeline:
   'Sign this random data with your private key to prove you own it!'


# Part 7: QL-Pipeline Signs the Challenge

QL-Pipeline uses its PRIVATE KEY to sign the challenge.

The private key never leaves QL-Pipeline's infrastructure!


In [23]:
print("\n" + "=" * 60)
print("PHASE 6: QL-PIPELINE SIGNS CHALLENGE")
print("=" * 60)

print("\n🔐 QL-Pipeline receives challenge")
print("   Challenge to sign:", challenge_b64[:40] + "...")

# QL-Pipeline signs the challenge with private key
print("\n🔏 Signing with PRIVATE KEY...")
print("   Using: RSA-PSS padding with SHA-256")


PHASE 6: QL-PIPELINE SIGNS CHALLENGE

🔐 QL-Pipeline receives challenge
   Challenge to sign: aiGYThslC95ErQKFqgcUn4Ys5cIf36RlVpD1pVJy...

🔏 Signing with PRIVATE KEY...
   Using: RSA-PSS padding with SHA-256


In [24]:
signature = ql_private_key.sign(
    challenge_data,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)

In [25]:
signature_b64 = base64.b64encode(signature).decode()
print(f"✅ Signature created: {len(signature)} bytes")
print(f"   Signature (base64): {signature_b64[:60]}...")

print("\n🔒 IMPORTANT: Private key was used but NEVER transmitted!")
print("   Only the SIGNATURE is sent to Provider A")

print("\n📤 QL-Pipeline → Provider A:")
print("   'Here is my signature of your challenge'")

✅ Signature created: 512 bytes
   Signature (base64): hFc35X+kdTEqePoyS4RE0p7kzghMEu4bkfx8lFNPiYpKM35K65J76mjwvWCz...

🔒 IMPORTANT: Private key was used but NEVER transmitted!
   Only the SIGNATURE is sent to Provider A

📤 QL-Pipeline → Provider A:
   'Here is my signature of your challenge'


# Part 8: Provider A Verifies the Signature

Provider A uses QL-Pipeline's PUBLIC KEY (from certificate) to verify the signature. 

If valid, it proves QL-Pipeline has the private key!


In [26]:
print("\n" + "=" * 60)
print("PHASE 7: PROVIDER A VERIFIES SIGNATURE")
print("=" * 60)

print("\n📥 Provider A receives signature from QL-Pipeline")
print(f"   Signature: {signature_b64[:60]}...")

print("\n🔍 Provider A verifies using PUBLIC KEY from certificate:")
print("   Challenge (original): ", challenge_b64[:40] + "...")
print("   Signature (received): ", signature_b64[:40] + "...")


PHASE 7: PROVIDER A VERIFIES SIGNATURE

📥 Provider A receives signature from QL-Pipeline
   Signature: hFc35X+kdTEqePoyS4RE0p7kzghMEu4bkfx8lFNPiYpKM35K65J76mjwvWCz...

🔍 Provider A verifies using PUBLIC KEY from certificate:
   Challenge (original):  aiGYThslC95ErQKFqgcUn4Ys5cIf36RlVpD1pVJy...
   Signature (received):  hFc35X+kdTEqePoyS4RE0p7kzghMEu4bkfx8lFNP...


In [28]:
# Provider A verifies signature with public key

try:
    provider_stored_public_key.verify(
        signature,
        challenge_data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    verification_result = "✅ VALID"
    print(f"\n{verification_result}")
    print("   The signature was created with QL-Pipeline's private key!")
    print("   Client is AUTHENTICATED as ql-pipeline.eu")
    
except Exception as e:
    verification_result = "❌ INVALID"
    print(f"\n{verification_result}")
    print(f"   Error: {e}")


✅ VALID
   The signature was created with QL-Pipeline's private key!
   Client is AUTHENTICATED as ql-pipeline.eu


# Part 9: What Happens with Invalid Signature?

Let's simulate an attacker trying to fake a signature.

In [29]:
print("\n" + "=" * 60)
print("DEMO: ATTACK SCENARIO - FAKE SIGNATURE")
print("=" * 60)

print("\n🦹 Attacker tries to access Provider A")
print("   Attacker has: ql-pipeline-client.crt (public certificate)")
print("   Attacker does NOT have: private key")

# Attacker tries to create fake signature
print("\n🎲 Provider A sends challenge to attacker")
attacker_challenge = os.urandom(32)

print("\n🦹 Attacker attempts to sign challenge:")
print("   Option 1: Random bytes as signature")
fake_signature = os.urandom(512)  # Random garbage

print(f"   Fake signature: {base64.b64encode(fake_signature).decode()[:60]}...")



DEMO: ATTACK SCENARIO - FAKE SIGNATURE

🦹 Attacker tries to access Provider A
   Attacker has: ql-pipeline-client.crt (public certificate)
   Attacker does NOT have: private key

🎲 Provider A sends challenge to attacker

🦹 Attacker attempts to sign challenge:
   Option 1: Random bytes as signature
   Fake signature: rGBWr/a5c1qO9BG9ZJONpLCrlBLR7ccoa5gXEYnhf//POwT1sPodCNwrNwsk...


In [30]:
# Provider A tries to verify

print("\n🔍 Provider A verifies attacker's signature:")
try:
    provider_stored_public_key.verify(
        fake_signature,
        attacker_challenge,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    print("   ✅ VALID (This should never happen!)")
except Exception as e:
    print("   ❌ INVALID SIGNATURE")
    print(f"   Error: Invalid signature")
    print("\n🚫 Provider A rejects connection!")
    print("   TLS Alert: decrypt_error")
    print("   Connection terminated")

print("\n🛡️ Result: Attacker BLOCKED!")
print("   Cannot create valid signature without private key")



🔍 Provider A verifies attacker's signature:
   ❌ INVALID SIGNATURE
   Error: Invalid signature

🚫 Provider A rejects connection!
   TLS Alert: decrypt_error
   Connection terminated

🛡️ Result: Attacker BLOCKED!
   Cannot create valid signature without private key


# Part 10: Successful Authentication - Data Transfer

After successful authentication, the secure channel is established.

In [31]:
print("\n" + "=" * 60)
print("PHASE 8: SECURE DATA TRANSFER")
print("=" * 60)

print("\n✅ mTLS Authentication Complete!")
print("   QL-Pipeline identity: VERIFIED")
print("   Secure channel: ESTABLISHED")
print("   Encryption: AES-256-GCM")

print("\n📡 Application-level communication begins:")
print("\n🔐 QL-Pipeline → Provider A:")
print("   GET /api/courses HTTP/1.1")
print("   Host: provider-a.edu")
print("   (Request is encrypted)")


PHASE 8: SECURE DATA TRANSFER

✅ mTLS Authentication Complete!
   QL-Pipeline identity: VERIFIED
   Secure channel: ESTABLISHED
   Encryption: AES-256-GCM

📡 Application-level communication begins:

🔐 QL-Pipeline → Provider A:
   GET /api/courses HTTP/1.1
   Host: provider-a.edu
   (Request is encrypted)


In [32]:
# Simulate course data

courses_data = {
    "courses": [
        {"id": "CS101", "name": "Introduction to Computer Science"},
        {"id": "CS201", "name": "Data Structures"},
        {"id": "MATH101", "name": "Calculus I"}
    ],
    "total": 3,
    "authenticated_client": "ql-pipeline.eu"
}

print("\n📤 Provider A → QL-Pipeline:")
print("   HTTP/1.1 200 OK")
print("   Content-Type: application/json")
print("   (Response is encrypted)")
print(f"\n   {courses_data}")

print("\n✅ Data successfully transferred!")
print("   Connection can be reused for subsequent requests")



📤 Provider A → QL-Pipeline:
   HTTP/1.1 200 OK
   Content-Type: application/json
   (Response is encrypted)

   {'courses': [{'id': 'CS101', 'name': 'Introduction to Computer Science'}, {'id': 'CS201', 'name': 'Data Structures'}, {'id': 'MATH101', 'name': 'Calculus I'}], 'total': 3, 'authenticated_client': 'ql-pipeline.eu'}

✅ Data successfully transferred!
   Connection can be reused for subsequent requests
